## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 3.1 MB/s 
     |████████████████████████████████| 6.3 MB 48.3 MB/s 
     |████████████████████████████████| 16.7 MB 164 kB/s 


In [10]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/GitHub/automating-gis-processes/Lesson-3/exercise-3-Nkumah7-main/data

/content/drive/MyDrive/GitHub/automating-gis-processes/Lesson-3/exercise-3-Nkumah7-main/data


In [38]:
# with open("activity_locations.txt", "a") as file:
#   file.write("id;addr\n")
#   file.write(f"{1};University of Liverpool, Liverpool, England\n")
#   file.write(f"{2};Liverpool Hope University, Hope Park, Liverpool, England\n")

In [36]:
# !rm activity_locations.txt

In [53]:
%cd ..

/content/drive/MyDrive/GitHub/automating-gis-processes/Lesson-3/exercise-3-Nkumah7-main


In [54]:
!ls

activity_locations.txt	      README.md		    shopping_centers.shp
data			      shopping_centers.cpg  shopping_centers.shx
Exercise-3-Problem-1-3.ipynb  shopping_centers.dbf  shopping_centers.txt
Exercise-3-Problem-4.ipynb    shopping_centers.prj


In [71]:
# Import modules
import os
import pygeos
import geojson
import requests
import pandas as pd
from pyproj import CRS
import geopandas as gpd
from geopandas.tools import geocode
from shapely.ops import nearest_points

In [55]:
# read in input files
data = pd.read_csv("activity_locations.txt", sep=";")

data.head()

,id,addr
0,0,"Koirasaarentie 35, 00590 Helsinki"
1,1,"Gustaf Hällströmin katu 2, 00560 Helsinki"


In [56]:
# gdf = gpd.GeoDataFrame(data, geometry='')

id       int64
addr    object
dtype: object

In [79]:
# Geocode activity locations

geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_jt', timeout=5)

In [80]:
geo.head()

,geometry,address
0,POINT (25.01811 60.16924),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta..."
1,POINT (24.96214 60.20501),"Kumpulan kampuskirjasto, 2, Gustaf Hällströmin..."


In [91]:
shops = gpd.read_file("shopping_centers.shp")
shops

,id,name,address,geometry
0,0,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",POINT (25504598.602 6677662.109)
1,1,Forum,"Salaattiasema, 14-20, Mannerheimintie, Keskust...",POINT (25496573.542 6672878.360)
2,2,Iso-Omena,"Captain Corvus, 11, Piispansilta, Matinkylä, S...",POINT (25485456.867 6671954.184)
3,3,Sello,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",POINT (25489491.076 6678322.265)
4,4,Jumbo,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",POINT (25497943.932 6686656.982)
5,5,REDI,"Silta, 5, Hermannin rantatie, Verkkosaari, Kal...",POINT (25498837.156 6674909.983)
6,6,Tripla,"Pasilansilta, Keski-Pasila, Pasila, Keskinen s...",POINT (25496361.661 6676135.584)


In [92]:
geo = geo.to_crs(epsg=3067)
shops = shops.to_crs(epsg=3067)

In [93]:
geo['centroid'] = geo.centroid
geo.head()

,geometry,address,centroid
0,POINT (390030.965 6671909.496),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",POINT (390030.965 6671909.496)
1,POINT (387048.749 6675986.268),"Kumpulan kampuskirjasto, 2, Gustaf Hällströmin...",POINT (387048.749 6675986.268)


In [88]:
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [94]:
unary_union = shops.unary_union
print(unary_union)

MULTIPOINT (374463.65264071885 6671406.411110206, 378687.9712300216 6677648.043688277, 385487.5978823316 6675254.792187851, 385600.71232322 6671993.427407321, 387387.5131471229 6685720.876568572, 387924.24619065877 6673955.093940117, 393764.9228955387 6676530.794747908)


In [95]:
geo["nearest_loc"] = geo.apply(get_nearest_values, other_gdf=shops, point_column="centroid", axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [96]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
geo["nearest_loc"] = geo.apply(get_nearest_values, other_gdf=shops, point_column="centroid", value_column="id", axis=1)

In [97]:
geo.head()

,geometry,address,centroid,nearest_loc
0,POINT (390030.965 6671909.496),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",POINT (390030.965 6671909.496),5
1,POINT (387048.749 6675986.268),"Kumpulan kampuskirjasto, 2, Gustaf Hällströmin...",POINT (387048.749 6675986.268),6


Can you think of other application cases for the nearest neighbour analysis?

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()